In [1]:
import nltk
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('punkt')

In [2]:
import re
import pandas as pd
from transformers import BertTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer




In [3]:


# Define the file path
file_path = "../data/costco-reviews.csv"  # Adjust path if needed

# Load the dataset
df = pd.read_csv(file_path)

# Display basic information
df.head()


,id,isVerified,lastModeratedTime,lastModificationTime,negativeFeedbackCount,positiveFeedbackCount,productId,productName,productUrl,rating,submissionTime,text,title,userLocation,userNickname
0,236287454,True,2024-09-20T18:15:11.000Z,2024-12-31T20:39:08.000Z,1,59,4000270696,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",https://www.costco.com/lg-77-class---oled-b4-s...,4,2024-09-20T12:15:09.000Z,The picture is beautiful. Four stars because ...,"Great picture, slow Ethernet, good wifi",NaN,Adam
1,238789353,True,2025-01-07T15:30:11.000Z,2025-01-22T18:12:02.000Z,0,3,4000270696,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",https://www.costco.com/lg-77-class---oled-b4-s...,5,2025-01-04T03:50:09.000Z,"I recently upgraded to the LG OLED 77” B4, and...",Stunning Picture Quality and Performance,NaN,JWITT
2,237195412,True,2024-11-03T10:15:05.000Z,2024-12-27T19:17:52.000Z,1,12,4000270696,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",https://www.costco.com/lg-77-class---oled-b4-s...,4,2024-11-03T10:06:58.000Z,"Great picture, but doesn’t play well in with Y...",Great picture OS app integration so so,SWF Florida,Radioactive
3,237204896,True,2024-11-04T00:45:10.000Z,2025-01-15T22:04:01.000Z,1,8,4000270696,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",https://www.costco.com/lg-77-class---oled-b4-s...,4,2024-11-03T17:22:31.000Z,"Costco was great about honoring the warranty, ...",Love it when it's working right,"Atlanta, GA",D Martin
4,239023152,True,2025-01-14T17:30:08.000Z,2025-01-22T18:11:40.000Z,0,1,4000270696,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",https://www.costco.com/lg-77-class---oled-b4-s...,5,2025-01-12T18:15:12.000Z,Sweet. Great picture and relatively easy to in...,Sweet.,Irving. Tx.,TYO1


In [4]:
df_no_duplicates = df.drop_duplicates()

In [5]:
# Select only the required columns
columns_to_keep = ["text", "title", "rating", "productName", "isVerified", "lastModificationTime","negativeFeedbackCount","productUrl", "positiveFeedbackCount","productId"]
df_nlp = df_no_duplicates[columns_to_keep]

# Display the first few rows of the new dataframe
df_nlp.head()

,text,title,rating,productName,isVerified,lastModificationTime,negativeFeedbackCount,productUrl,positiveFeedbackCount,productId
0,The picture is beautiful. Four stars because ...,"Great picture, slow Ethernet, good wifi",4,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",True,2024-12-31T20:39:08.000Z,1,https://www.costco.com/lg-77-class---oled-b4-s...,59,4000270696
1,"I recently upgraded to the LG OLED 77” B4, and...",Stunning Picture Quality and Performance,5,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",True,2025-01-22T18:12:02.000Z,0,https://www.costco.com/lg-77-class---oled-b4-s...,3,4000270696
2,"Great picture, but doesn’t play well in with Y...",Great picture OS app integration so so,4,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",True,2024-12-27T19:17:52.000Z,1,https://www.costco.com/lg-77-class---oled-b4-s...,12,4000270696
3,"Costco was great about honoring the warranty, ...",Love it when it's working right,4,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",True,2025-01-15T22:04:01.000Z,1,https://www.costco.com/lg-77-class---oled-b4-s...,8,4000270696
4,Sweet. Great picture and relatively easy to in...,Sweet.,5,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",True,2025-01-22T18:11:40.000Z,0,https://www.costco.com/lg-77-class---oled-b4-s...,1,4000270696


In [6]:
df_nlp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 126495 entries, 0 to 126517
Data columns (total 10 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   text                   126422 non-null  object
 1   title                  125023 non-null  object
 2   rating                 126495 non-null  int64 
 3   productName            126495 non-null  object
 4   isVerified             126495 non-null  bool  
 5   lastModificationTime   126495 non-null  object
 6   negativeFeedbackCount  126495 non-null  int64 
 7   productUrl             126495 non-null  object
 8   positiveFeedbackCount  126495 non-null  int64 
 9   productId              126495 non-null  int64 
dtypes: bool(1), int64(4), object(5)
memory usage: 9.8+ MB


In [7]:
df_nlp['isVerified'].value_counts()


isVerified
True     120037
False      6458
Name: count, dtype: int64

In [8]:
df_nlp = df_nlp[df_nlp['isVerified'] == True]
df_nlp['isVerified'].value_counts()

isVerified
True    120037
Name: count, dtype: int64

In [9]:
df_nlp.sort_values(by='negativeFeedbackCount', ascending=False).head(15)



,text,title,rating,productName,isVerified,lastModificationTime,negativeFeedbackCount,productUrl,positiveFeedbackCount,productId
1840,The TV is 61” the promotion said 70” very d...,Tv under 70”,1,"Samsung 70"" Class - DU6900D Series - 4K UHD LE...",True,2025-01-31T19:11:13.000Z,1198,https://www.costco.com/samsung-70-class---du69...,33,4000304064
61435,TV arrived and costco logistic were best but T...,Honest review,1,"LG 65"" Class - UR8000 Series - 4K UHD LED LCD TV",True,2025-01-30T16:17:33.000Z,978,https://www.costco.com/lg-86-class---ur8000-se...,561,4000153847
63728,I bought the Queen size and loved it so I then...,Too soft!!,1,"Novaform 14"" ComfortGrande Plus Gel Memory Foa...",True,2025-01-26T16:42:57.000Z,460,https://www.costco.com/novaform-14-comfortgran...,870,100667300
47099,After seeing there were 102 pages of instructi...,Complicated assembly.,1,Yardistry 8’ x 7' Greenhouse,True,2025-01-30T13:44:37.000Z,372,https://www.costco.com/yardistry-8-x-7-greenho...,178,100784646
14400,Buyer beware! There are two different colors ...,Two different shades of grey,1,Thomasville Artesia Corduroy Fabric Sectional ...,True,2024-12-05T14:51:44.000Z,292,https://www.costco.com/thomasville-artesia-cor...,286,4000213937
98841,I put this together easily with another person...,Excellent Quality. Zero Instructions,1,Centr 2 by Inspire FTX Functional Trainer with...,True,2023-10-07T16:27:07.000Z,292,https://www.costco.com/centr-2-by-inspire-ftx-...,84,4000212689
23232,Missing from specifications and description of...,MAGNETS WILL NOT STICK TO THE FRONT!,2,LG 23.8 cu. ft. Top Mount Refrigerator with In...,True,2025-01-30T17:04:30.000Z,236,https://www.costco.com/lg-238-cu-ft-top-mount-...,72,4000153101
46764,I have done more research on beds then any one...,Great price!,4,"Kirkland Signature by Stearns & Foster 14.5"" L...",True,2025-01-31T21:43:01.000Z,233,https://www.costco.com/kirkland-signature-by-s...,4996,100541532
64080,Purchased the delivery and install. The guys j...,So y tv delivery and I stall,1,"Sony 65"" Class - X90CL Series - 4K UHD LED LCD...",True,2024-12-12T06:47:52.000Z,225,https://www.costco.com/sony-85-class---x90cl-s...,48,4000186628
7083,The delivery service was very incompetent. The...,"Great Tv, bad delivery service",2,"Hisense 100"" Class - U7 Series - 4K UHD QLED L...",True,2025-01-31T23:55:00.000Z,217,https://www.costco.com/hisense-100-class---u7-...,249,4000285099


In [10]:
df_nlp = df_nlp[df_nlp['negativeFeedbackCount'] <= 1]


In [11]:
df_nlp.sort_values(by='negativeFeedbackCount', ascending=False).head(15)



,text,title,rating,productName,isVerified,lastModificationTime,negativeFeedbackCount,productUrl,positiveFeedbackCount,productId
126471,My wife and I like the product look; however t...,Disgruntled Customer,1,Thomasville Tisdale Modular Sectional 6-piece ...,True,2025-01-26T16:12:57.000Z,1,https://www.costco.com/thomasville-tisdale-cor...,1,4000242730
126422,Beautiful set. Wish the seat cushions can be ...,New sofa,4,Thomasville Tisdale Modular Sectional 6-piece ...,True,2024-07-05T01:22:48.000Z,1,https://www.costco.com/thomasville-tisdale-cor...,2,4000242730
126310,This firm mattress is of a poor quality. Afte...,Not a great firm mattress.,2,Sealy Posturepedic Carver 11” Firm or 13.5” Pl...,True,2023-03-09T01:43:45.000Z,1,https://www.costco.com/sealy-posturepedic-carv...,1,100694157
126301,We got the firm mattress and it is very comfor...,Great mattress - comfortable,5,Sealy Posturepedic Carver 11” Firm or 13.5” Pl...,True,2021-09-13T06:29:39.000Z,1,https://www.costco.com/sealy-posturepedic-carv...,2,100694157
126287,Ordered it for our Guest Room and have already...,Ultra-Comfort,5,Sealy Posturepedic Carver 11” Firm or 13.5” Pl...,True,2022-12-28T02:48:33.000Z,1,https://www.costco.com/sealy-posturepedic-carv...,0,100694157
126254,I’ve choosen firm type considering my back pai...,No comform and Fake Product,1,Sealy Posturepedic Carver 11” Firm or 13.5” Pl...,True,2023-08-27T13:45:26.000Z,1,https://www.costco.com/sealy-posturepedic-carv...,2,100694157
126181,We read the reviews that said this came too fi...,Too soft and not enough support,1,"Novaform 14"" Serafina Pearl Gel Memory Foam Ma...",True,2021-11-20T01:57:25.000Z,1,https://www.costco.com/novaform-14-serafina-pe...,3,100661644
126175,"Fine, not great, mattress. It's been a month a...","Mediocre Mattress, Terrible Delivery",1,"Novaform 14"" Serafina Pearl Gel Memory Foam Ma...",True,2021-11-20T02:07:02.000Z,1,https://www.costco.com/novaform-14-serafina-pe...,0,100661644
126166,This mattress was not firm at all. Not very c...,NaN,1,"Novaform 14"" Serafina Pearl Gel Memory Foam Ma...",True,2022-08-26T20:27:26.000Z,1,https://www.costco.com/novaform-14-serafina-pe...,4,100661644
126157,Three tries to get the mattress right. First c...,Third time,4,"Novaform 14"" Serafina Pearl Gel Memory Foam Ma...",True,2024-02-21T13:45:00.000Z,1,https://www.costco.com/novaform-14-serafina-pe...,3,100661644


In [12]:
df_nlp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 113550 entries, 0 to 126517
Data columns (total 10 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   text                   113501 non-null  object
 1   title                  112614 non-null  object
 2   rating                 113550 non-null  int64 
 3   productName            113550 non-null  object
 4   isVerified             113550 non-null  bool  
 5   lastModificationTime   113550 non-null  object
 6   negativeFeedbackCount  113550 non-null  int64 
 7   productUrl             113550 non-null  object
 8   positiveFeedbackCount  113550 non-null  int64 
 9   productId              113550 non-null  int64 
dtypes: bool(1), int64(4), object(5)
memory usage: 8.8+ MB


In [13]:
df_nlp = df_nlp.dropna(subset=['text'])  # Remove rows where 'text' is missing
df_nlp['title'] = df_nlp['title'].fillna('')  # Fill missing 'title' with empty string


In [14]:
df_nlp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 113501 entries, 0 to 126517
Data columns (total 10 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   text                   113501 non-null  object
 1   title                  113501 non-null  object
 2   rating                 113501 non-null  int64 
 3   productName            113501 non-null  object
 4   isVerified             113501 non-null  bool  
 5   lastModificationTime   113501 non-null  object
 6   negativeFeedbackCount  113501 non-null  int64 
 7   productUrl             113501 non-null  object
 8   positiveFeedbackCount  113501 non-null  int64 
 9   productId              113501 non-null  int64 
dtypes: bool(1), int64(4), object(5)
memory usage: 8.8+ MB


In [15]:
df_updated = df_nlp.drop(columns=['title','isVerified', 'productId','negativeFeedbackCount'])


In [16]:
df_updated.head()

,text,rating,productName,lastModificationTime,productUrl,positiveFeedbackCount
0,The picture is beautiful. Four stars because ...,4,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",2024-12-31T20:39:08.000Z,https://www.costco.com/lg-77-class---oled-b4-s...,59
1,"I recently upgraded to the LG OLED 77” B4, and...",5,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",2025-01-22T18:12:02.000Z,https://www.costco.com/lg-77-class---oled-b4-s...,3
2,"Great picture, but doesn’t play well in with Y...",4,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",2024-12-27T19:17:52.000Z,https://www.costco.com/lg-77-class---oled-b4-s...,12
3,"Costco was great about honoring the warranty, ...",4,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",2025-01-15T22:04:01.000Z,https://www.costco.com/lg-77-class---oled-b4-s...,8
4,Sweet. Great picture and relatively easy to in...,5,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",2025-01-22T18:11:40.000Z,https://www.costco.com/lg-77-class---oled-b4-s...,1


In [17]:
df_updated.groupby('productName')['rating'].apply(lambda x: (x >= 4).sum())


productName
10' X 20' Steel Frame Canopy                                            126
93" Plush Bear, Blonde                                                   71
Agio - McKinnon 5-piece Fire Outdoor Seating Set                        111
Agio Bridgeport 4-piece Outdoor Patio Seating Set                       720
Agio Cullman 9-piece Sling Outdoor Patio Dining Set                     138
                                                                       ... 
Yardistry 12' x 20' Gazebo with Aluminum Roof                           300
Yardistry 12’ x 14’ Gazebo with Aluminum Roof                           130
Yardistry 8' x 10' Greenhouse                                           143
Yardistry 8’ x 7' Greenhouse                                            449
Zakary 6-piece Fabric Power Reclining Sectional With Power Headrests     14
Name: rating, Length: 659, dtype: int64

In [18]:
df_updated.groupby('productName')['rating'].apply(lambda x: (x >= 4).sum()).reset_index(name='likes').head()


,productName,likes
0,10' X 20' Steel Frame Canopy,126
1,"93"" Plush Bear, Blonde",71
2,Agio - McKinnon 5-piece Fire Outdoor Seating Set,111
3,Agio Bridgeport 4-piece Outdoor Patio Seating Set,720
4,Agio Cullman 9-piece Sling Outdoor Patio Dinin...,138


In [19]:
# Group by productName and count likes based on rating >= 4
# Calculate likes for each product
likes_count = df_updated.groupby('productName')['rating'].apply(lambda x: (x >= 4).sum()).reset_index(name='likes')

In [20]:
# Merge likes count back to the original DataFrame
df_updated = df_updated.merge(likes_count, on='productName', how='left')

In [21]:
df_updated.head()

,text,rating,productName,lastModificationTime,productUrl,positiveFeedbackCount,likes
0,The picture is beautiful. Four stars because ...,4,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",2024-12-31T20:39:08.000Z,https://www.costco.com/lg-77-class---oled-b4-s...,59,94
1,"I recently upgraded to the LG OLED 77” B4, and...",5,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",2025-01-22T18:12:02.000Z,https://www.costco.com/lg-77-class---oled-b4-s...,3,94
2,"Great picture, but doesn’t play well in with Y...",4,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",2024-12-27T19:17:52.000Z,https://www.costco.com/lg-77-class---oled-b4-s...,12,94
3,"Costco was great about honoring the warranty, ...",4,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",2025-01-15T22:04:01.000Z,https://www.costco.com/lg-77-class---oled-b4-s...,8,94
4,Sweet. Great picture and relatively easy to in...,5,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",2025-01-22T18:11:40.000Z,https://www.costco.com/lg-77-class---oled-b4-s...,1,94


In [22]:
# Split lastModificationTime into comment_year and comment_month
df_updated['comment_year'] = pd.to_datetime(df_updated['lastModificationTime']).dt.year
df_updated['comment_month'] = pd.to_datetime(df_updated['lastModificationTime']).dt.month
df_updated.head()

,text,rating,productName,lastModificationTime,productUrl,positiveFeedbackCount,likes,comment_year,comment_month
0,The picture is beautiful. Four stars because ...,4,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",2024-12-31T20:39:08.000Z,https://www.costco.com/lg-77-class---oled-b4-s...,59,94,2024,12
1,"I recently upgraded to the LG OLED 77” B4, and...",5,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",2025-01-22T18:12:02.000Z,https://www.costco.com/lg-77-class---oled-b4-s...,3,94,2025,1
2,"Great picture, but doesn’t play well in with Y...",4,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",2024-12-27T19:17:52.000Z,https://www.costco.com/lg-77-class---oled-b4-s...,12,94,2024,12
3,"Costco was great about honoring the warranty, ...",4,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",2025-01-15T22:04:01.000Z,https://www.costco.com/lg-77-class---oled-b4-s...,8,94,2025,1
4,Sweet. Great picture and relatively easy to in...,5,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",2025-01-22T18:11:40.000Z,https://www.costco.com/lg-77-class---oled-b4-s...,1,94,2025,1


In [23]:
# Add two new columns: category and source
df_updated['category'] = 'Product'
df_updated['source'] = 'Costco'

In [24]:
df_updated.head()

,text,rating,productName,lastModificationTime,productUrl,positiveFeedbackCount,likes,comment_year,comment_month,category,source
0,The picture is beautiful. Four stars because ...,4,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",2024-12-31T20:39:08.000Z,https://www.costco.com/lg-77-class---oled-b4-s...,59,94,2024,12,Product,Costco
1,"I recently upgraded to the LG OLED 77” B4, and...",5,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",2025-01-22T18:12:02.000Z,https://www.costco.com/lg-77-class---oled-b4-s...,3,94,2025,1,Product,Costco
2,"Great picture, but doesn’t play well in with Y...",4,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",2024-12-27T19:17:52.000Z,https://www.costco.com/lg-77-class---oled-b4-s...,12,94,2024,12,Product,Costco
3,"Costco was great about honoring the warranty, ...",4,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",2025-01-15T22:04:01.000Z,https://www.costco.com/lg-77-class---oled-b4-s...,8,94,2025,1,Product,Costco
4,Sweet. Great picture and relatively easy to in...,5,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",2025-01-22T18:11:40.000Z,https://www.costco.com/lg-77-class---oled-b4-s...,1,94,2025,1,Product,Costco


In [25]:
# Drop the 'rating' and 'lastModificationTime' columns
df_updated = df_updated.drop(columns=['rating', 'lastModificationTime'])
df_updated.head()

,text,productName,productUrl,positiveFeedbackCount,likes,comment_year,comment_month,category,source
0,The picture is beautiful. Four stars because ...,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",https://www.costco.com/lg-77-class---oled-b4-s...,59,94,2024,12,Product,Costco
1,"I recently upgraded to the LG OLED 77” B4, and...","LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",https://www.costco.com/lg-77-class---oled-b4-s...,3,94,2025,1,Product,Costco
2,"Great picture, but doesn’t play well in with Y...","LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",https://www.costco.com/lg-77-class---oled-b4-s...,12,94,2024,12,Product,Costco
3,"Costco was great about honoring the warranty, ...","LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",https://www.costco.com/lg-77-class---oled-b4-s...,8,94,2025,1,Product,Costco
4,Sweet. Great picture and relatively easy to in...,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",https://www.costco.com/lg-77-class---oled-b4-s...,1,94,2025,1,Product,Costco


In [26]:
df_updated = df_updated.rename(columns={
    'text': 'comment_body',
    'productName': 'post_title',
    'productUrl': 'post_url',
    'positiveFeedbackCount': 'comment_likes'
})

# Reorder columns
df_updated = df_updated[
    ['post_title', 'post_url', 'likes', 'comment_body', 'comment_likes', 'comment_year', 'comment_month', 'category', 'source']
]

# Display the updated DataFrame
df_updated.head()

,post_title,post_url,likes,comment_body,comment_likes,comment_year,comment_month,category,source
0,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",https://www.costco.com/lg-77-class---oled-b4-s...,94,The picture is beautiful. Four stars because ...,59,2024,12,Product,Costco
1,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",https://www.costco.com/lg-77-class---oled-b4-s...,94,"I recently upgraded to the LG OLED 77” B4, and...",3,2025,1,Product,Costco
2,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",https://www.costco.com/lg-77-class---oled-b4-s...,94,"Great picture, but doesn’t play well in with Y...",12,2024,12,Product,Costco
3,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",https://www.costco.com/lg-77-class---oled-b4-s...,94,"Costco was great about honoring the warranty, ...",8,2025,1,Product,Costco
4,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",https://www.costco.com/lg-77-class---oled-b4-s...,94,Sweet. Great picture and relatively easy to in...,1,2025,1,Product,Costco


In [27]:
df_updated.to_csv('prepped.csv', index=False)

In [28]:
df_updated['post_title'] = df_updated['post_title'].fillna('')

In [29]:
print(df_updated.iloc[0]['comment_body'])

The picture is beautiful.  Four stars because of the Ethernet port being much slower than wifi, going against conventional logic.  After being on the phone with LG and the internet provider confirming adequate speeds, I finally found multiple blogs discussing how LG had used very low quality Ethernet hardware that was not capable of equivalent wifi speeds.  I wanted to be able to connect to a hard line for steady streaming, but when connected, streaming would stutter and buffer, apps wouldn't load, and when I disconnected and let wifi take over everything worked smoothly as expected. I am including this information because of the time I had to spend on the phone with LG and Verizon... just check the blogs and forums on line and it seems the common people figured out the issue.


In [30]:
# Clean text (skip post_title)
def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\\S+', '', text)  # Remove URLs
    # Use raw string to retain spaces correctly
    text = re.sub(r'[^a-zA-Z0-9.,!?$&@#\s]', '', text)  # Remove only unnecessary special characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

df_updated['comment_body'] = df_updated['comment_body'].apply(clean_text)

In [31]:
print(df_updated.iloc[0]['comment_body'])

the picture is beautiful. four stars because of the ethernet port being much slower than wifi, going against conventional logic. after being on the phone with lg and the internet provider confirming adequate speeds, i finally found multiple blogs discussing how lg had used very low quality ethernet hardware that was not capable of equivalent wifi speeds. i wanted to be able to connect to a hard line for steady streaming, but when connected, streaming would stutter and buffer, apps wouldnt load, and when i disconnected and let wifi take over everything worked smoothly as expected. i am including this information because of the time i had to spend on the phone with lg and verizon... just check the blogs and forums on line and it seems the common people figured out the issue.


In [32]:
#Remove stopwords
stop_words = set(stopwords.words('english'))
df_updated['comment_body'] = df_updated['comment_body'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

In [33]:
print(df_updated.iloc[0]['comment_body'])

picture beautiful. four stars ethernet port much slower wifi, going conventional logic. phone lg internet provider confirming adequate speeds, finally found multiple blogs discussing lg used low quality ethernet hardware capable equivalent wifi speeds. wanted able connect hard line steady streaming, connected, streaming would stutter buffer, apps wouldnt load, disconnected let wifi take everything worked smoothly expected. including information time spend phone lg verizon... check blogs forums line seems common people figured issue.


In [34]:
# Lemmatization
lemmatizer = WordNetLemmatizer()
df_updated['comment_body'] = df_updated['comment_body'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

In [35]:
print(df_updated.iloc[0]['comment_body'])

picture beautiful. four star ethernet port much slower wifi, going conventional logic. phone lg internet provider confirming adequate speeds, finally found multiple blog discussing lg used low quality ethernet hardware capable equivalent wifi speeds. wanted able connect hard line steady streaming, connected, streaming would stutter buffer, apps wouldnt load, disconnected let wifi take everything worked smoothly expected. including information time spend phone lg verizon... check blog forum line seems common people figured issue.


In [36]:
# Tokenization using BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
df_updated['tokens'] = df_updated['comment_body'].apply(lambda x: tokenizer.tokenize(x))



In [37]:
print(df_updated.iloc[0]['tokens'])

['picture', 'beautiful', '.', 'four', 'star', 'ethernet', 'port', 'much', 'slower', 'wi', '##fi', ',', 'going', 'conventional', 'logic', '.', 'phone', 'l', '##g', 'internet', 'provider', 'confirming', 'adequate', 'speeds', ',', 'finally', 'found', 'multiple', 'blog', 'discussing', 'l', '##g', 'used', 'low', 'quality', 'ethernet', 'hardware', 'capable', 'equivalent', 'wi', '##fi', 'speeds', '.', 'wanted', 'able', 'connect', 'hard', 'line', 'steady', 'streaming', ',', 'connected', ',', 'streaming', 'would', 'stu', '##tter', 'buffer', ',', 'apps', 'wouldn', '##t', 'load', ',', 'disconnected', 'let', 'wi', '##fi', 'take', 'everything', 'worked', 'smoothly', 'expected', '.', 'including', 'information', 'time', 'spend', 'phone', 'l', '##g', 've', '##riz', '##on', '.', '.', '.', 'check', 'blog', 'forum', 'line', 'seems', 'common', 'people', 'figured', 'issue', '.']


In [38]:
# Display the preprocessed DataFrame
display(df_updated.head())

,post_title,post_url,likes,comment_body,comment_likes,comment_year,comment_month,category,source,tokens
0,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",https://www.costco.com/lg-77-class---oled-b4-s...,94,picture beautiful. four star ethernet port muc...,59,2024,12,Product,Costco,"[picture, beautiful, ., four, star, ethernet, ..."
1,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",https://www.costco.com/lg-77-class---oled-b4-s...,94,"recently upgraded lg oled 77 b4, completely tr...",3,2025,1,Product,Costco,"[recently, upgraded, l, ##g, ole, ##d, 77, b, ..."
2,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",https://www.costco.com/lg-77-class---oled-b4-s...,94,"great picture, doesnt play well tube tv app. v...",12,2024,12,Product,Costco,"[great, picture, ,, doesn, ##t, play, well, tu..."
3,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",https://www.costco.com/lg-77-class---oled-b4-s...,94,"costco great honoring warranty, good me, 1st l...",8,2025,1,Product,Costco,"[cost, ##co, great, honoring, warrant, ##y, ,,..."
4,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",https://www.costco.com/lg-77-class---oled-b4-s...,94,sweet. great picture relatively easy install. ...,1,2025,1,Product,Costco,"[sweet, ., great, picture, relatively, easy, i..."


In [39]:
df_updated.to_csv('Costco.csv', index=False)